In [6]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")

import numpy as np
import math
import pandas as pd
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import calibrate_power as cbp

from scipy import signal
from read_scope_data import read_trc_data

%matplotlib widget

In [7]:
def read_excel_sheets(filepath):
    """
    Read an Excel file and extract columns with non-zero values from each sheet.

    Parameters:
    filepath (str): The path to the Excel file.

    Returns:
    dict: A dictionary containing the sheet names as keys and the columns with non-zero values as values.
    """
    data_dict = {}

    # Read the Excel file
    xls = pd.ExcelFile(filepath)

    # Iterate over each sheet in the Excel file
    for sheet_name in xls.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(filepath, sheet_name=sheet_name)

        # Select the columns with non-zero values from the DataFrame
        non_zero_columns = df.loc[:, (df != 0).any(axis=0)].values
        print (non_zero_columns.shape)
        # Add the non-zero columns to the data dictionary
        #data_dict[sheet_name] = non_zero_columns

    xls.close()

    return data_dict

ifn = r"C:\data\epfl\diagnostic source\CRDS\solenoid\20230531_1mTorr_0.012lnmin.xlsx"
data = read_excel_sheets(ifn)

'''
def save_to_npy(filepath, columns, skip_rows=0):
    # Check if the .npy file already exists
    npy_file_path = f"{sheet_name}.npy"
    if os.path.exists(npy_file_path):
        # Load the existing data dictionary from the .npy file
        existing_data_dict = np.load(npy_file_path, allow_pickle=True).item()
        # Append the selected columns to the existing data dictionary
        existing_data_dict[sheet_name] = pd.to_numeric(selected_columns, errors='coerce').dropna()
        # Save the updated data dictionary back to the .npy file
        np.save(npy_file_path, existing_data_dict)
    else:
        # Create a new data dictionary and save it as a .npy file
        data_dict[sheet_name] = pd.to_numeric(selected_columns, errors='coerce').dropna()
        np.save(npy_file_path, data_dict)

    return npy_file_path
'''

(13616, 6)
(9535, 6)
(11630, 6)
(25142, 15)
(9856, 6)


'\ndef save_to_npy(filepath, columns, skip_rows=0):\n    # Check if the .npy file already exists\n    npy_file_path = f"{sheet_name}.npy"\n    if os.path.exists(npy_file_path):\n        # Load the existing data dictionary from the .npy file\n        existing_data_dict = np.load(npy_file_path, allow_pickle=True).item()\n        # Append the selected columns to the existing data dictionary\n        existing_data_dict[sheet_name] = pd.to_numeric(selected_columns, errors=\'coerce\').dropna()\n        # Save the updated data dictionary back to the .npy file\n        np.save(npy_file_path, existing_data_dict)\n    else:\n        # Create a new data dictionary and save it as a .npy file\n        data_dict[sheet_name] = pd.to_numeric(selected_columns, errors=\'coerce\').dropna()\n        np.save(npy_file_path, data_dict)\n\n    return npy_file_path\n'

In [ ]:
'''
Some C1 records antenna current, code commented out included as needed
ifn = r"C:\data\epfl\CRDS\scope trc\bdcageAnt\C1-300W-1mT-00000.trc"
Iant, tarr = read_trc_data(ifn)
Iant *= 8 # Pearson 8A/V
'''

def read_crds_data(ifn):
    ifn_light = r"C:\data\epfl\CRDS\scope trc\solAnt\C3-500W-36mT-00000.trc"

    light, tarr = read_trc_data(ifn)
    print(tarr.shape, light.shape)

    ifn_Vtrig = r"C:\data\epfl\CRDS\scope trc\solAnt\C4-500W-36mT-00000.trc"
    Vtrig, tarr = read_trc_data(ifn)

    fig, ax = plt.subplots(nrows=3, sharex=True)

    ax[1].plot(tarr*1e3, light)
    ax[1].set_ylabel('Light')

    ax[2].plot(tarr*1e3, Vtrig)
    ax[2].set_ylabel('TTL')
    ax[2].set_xlabel('time (ms)')

    start = 22000
    stop = 24000
    ax[1].plot(tarr[start:stop]*1e3, light[start:stop]) # plot range to take average
    lavg = np.average(light[start:stop]) * 1e3 # take average light signal conver V to mV
    print(cbp.ltop_solenoid(lavg, 35)) # Calibrated power according to CW light signal

    ifn_crds =  #r"C:\data\epfl\CRDS\solenoid\20230601_0.5mTorr_0.007lnmin.xlsx"

    # xlsx is only supported by openpyxl special package because they saved the excel on a very old version ugh
    excel_data_df = pd.read_excel(ifn, sheet_name=0, engine='openpyxl', header=[0])

    hdr_ls = excel_data_df.columns.ravel()
    print(hdr_ls)
    tarr_crds = excel_data_df[hdr_ls[0]].to_numpy()
    sig_crds = excel_data_df[hdr_ls[1]].to_numpy()

    tarr_crds = tarr_crds[~np.isnan(sig_crds)]
    sig_crds = sig_crds[~np.isnan(sig_crds)]

    print(len(sig_crds))

    return tarr_crds, sig_crds

In [ ]:
fig = plt.figure()
plt.scatter(tarr_crds, sig_crds)

# signal baseline
start = 200
stop = 240
plt.scatter(tarr_crds[start:stop], sig_crds[start:stop])
sig_base = np.average(sig_crds[start:stop])

# signal active-glow equilibrium value
start = -70
stop = -20
plt.scatter(tarr_crds[start:stop], sig_crds[start:stop])
sig_val = np.average(sig_crds[start:stop])

print('%.2e' %(sig_val-sig_base))

In [ ]:
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(nrows=3, sharex=True)
fig.suptitle('5mTorr/0.67Pa 580W')

ax[0].plot(tarr*1e3, light)
ax[0].set_ylabel('Light')

ax[1].plot(tarr*1e3, Vtrig)
ax[1].set_ylabel('TTL')

tarr_fake = tarr_crds + tarr_crds[-1]
tarr_crds_mod = np.hstack((tarr_crds, tarr_fake)) #+tarr_crds[100]
sig_crds_mod = np.tile(sig_crds , 2) #numpy.roll(sig_crds,-100)

ax[2].plot(tarr_crds_mod*1e3, sig_crds_mod)
ax[2].set_ylabel('crds signal')
ax[2].set_xlabel('time (ms)')